In [1]:
import pandas as pd
import numpy as np

In [15]:
#Read in csv file
df = pd.read_csv("election_data.csv")

#Filter for 2012 and 2016
df_2012 = df[df["year"] == 2012]
df_2016 = df[df["year"] == 2016]

In [19]:
#Create pivot table with candidates as columns and votes as vales
pivot_2012 = df_2012.pivot(columns = "candidate", values = "candidatevotes")

#Merge pivot table with original 2012 dataframe
merged_2012 = df_2012.merge(pivot_2012, right_index=True, left_index=True)

#Group by state and county and sum (this allows all votes to be on same row)
merged_grouped_2012 = merged_2012.groupby(["state_po", "county"]).sum()

#Select last three rows and transform to ints
election_2012 = merged_grouped_2012.iloc[:,-3:].astype(int)

#Create total votes column
election_2012["Total Votes"] = election_2012.sum(1)

#Show results
election_2012.head(10)


Barack Obama  Mitt Romney  Other  Total Votes
state_po county                                                    
AK       District 1           1518         5899    305         7722
         District 10          1344         4928    298         6570
         District 11          2222         6057    352         8631
         District 12          1901         4300    281         6482
         District 13          2425         3078    204         5707
         District 14          3340         3453    275         7068
         District 15          3283         2875    293         6451
         District 16          4013         2876    292         7181
         District 17          2853         1542    194         4589
         District 18          4247         2369    314         6930

In [20]:
#Create pivot table with candidates as columns and votes as vales
pivot_2016 = df_2016.pivot(columns = "candidate", values = "candidatevotes")

#Merge pivot table with original 2016 dataframe
merged_2016 = df_2016.merge(pivot_2016, right_index=True, left_index=True)

#Group by state and county and sum (this allows all votes to be on same row)
merged_grouped_2016 = merged_2016.groupby(["state_po", "county"]).sum()

#Select last three rows and transform to ints
election_2016 = merged_grouped_2016.iloc[:,-3:].astype(int)

#Create total votes column
election_2016["Total Votes"] = election_2016.sum(1)

#Show results
election_2016.head(10)



Donald Trump  Hillary Clinton  Other  Total Votes
state_po county                                                        
AK       District 1           3180             2573    885         6638
         District 10          6255             1808    977         9040
         District 11          6444             2142   1103         9689
         District 12          6629             1928    986         9543
         District 13          4028             1684    821         6533
         District 14          5978             3043   1399        10420
         District 15          2525             1828    629         4982
         District 16          3203             3294    939         7436
         District 17          2618             3290    880         6788
         District 18          2684             3909    809         7402

In [22]:
#Join 2012 and 2016 dataframes
election_df = election_2012.join(election_2016, rsuffix = "_2016", lsuffix="_2012")

#Show results
election_df.head(10)

Barack Obama  Mitt Romney  Other_2012  Total Votes_2012  \
state_po county                                                                 
AK       District 1           1518         5899         305              7722   
         District 10          1344         4928         298              6570   
         District 11          2222         6057         352              8631   
         District 12          1901         4300         281              6482   
         District 13          2425         3078         204              5707   
         District 14          3340         3453         275              7068   
         District 15          3283         2875         293              6451   
         District 16          4013         2876         292              7181   
         District 17          2853         1542         194              4589   
         District 18          4247         2369         314              6930   

                      Donald Trump  Hillary Clinton  Other_2016  \
state_po county                                                   
AK       District 1           3180             2573         885   
         District 10          6255             1808         977   
         District 11          6444             2142        1103   
         District 12          6629             1928         986   
         District 13          4028             1684         821   
         District 14          5978             3043        1399   
         District 15          2525             1828         629   
         District 16          3203             3294         939   
         District 17          2618             3290         880   
         District 18          2684             3909         809   

                      Total Votes_2016  
state_po county                         
AK       District 1               6638  
         District 10              9040  
         District 11              9689  
         District 12              9543  
         District 13              6533  
         District 14             10420  
         District 15              4982  
         District 16              7436  
         District 17              6788  
         District 18              7402